In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import SimpleITK as sitk

import os
import time
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    from tqdm import tqdm_notebook as tqdm
    haveTQDM = True
except:
    haveTQDM = False
    
print(haveTQDM)
    
%matplotlib inline

## Download the data

NB: you might not need to do this!

In [ ]:
!wget https://www.dropbox.com/s/ec8y5vb8frdzhfc/HNSCC.zip?dl=0 -O ./HNSCC.zip
!unzip -q HNSCC.zip
!rm HNSCC.zip

## Getting started

The initial setup can be largely the same as in the other notebooks, lets start by reading the clinical data and processing it in a slightly better way

In [ ]:
## Load the clinical data CSV
clinicalDataPath = "./HNSCC/clinicalData.csv"

clinicalData = pd.read_csv(clinicalDataPath)



In contrast to the last analysis, we are not going to filter patients based on their treatment. Instead, we will include some of their clinical data in the analysis as additional covariates. This will make the analysis a bit trickier, but should mean we have a more robust answer at the end.

We will still exlude the 40 fraction patients, as they have an unusual treatment regimen, but otherwise we will use the same BED calculation as before


In [ ]:
## This function should create a voxelwise BED distribution. It makes the assumption that the dose in the plan is delivered in equal fractions
## This is obviously not 100% true due to setup uncertainty, motion, etc, but it is an ok assumtion for now

## BED = D_t * (1 + D_f/ab)  where:
## D_t is the total dose (per voxel)
## D_f = D_t/nFrac is the fraction dose per voxel
## ab is the alpha/beta ratio

def calculateVoxelwiseBED(dose, nFrac, alpha_beta=10.0):
    factor = 1.0 / (nFrac*alpha_beta)

    BED = dose*(1.0 + dose*factor)

    return BED

In [ ]:
## Take all comers, but exclude 40 fraction patients

selectedPatients = clinicalData[clinicalData["Number of Fractions"].astype(int) < 40]
len(selectedPatients["Number of Fractions"])

In terms of outcome, we will still look at survival 12 months post radiotherapy, so we can copy the dose loading and outcome specification but from the simpler binary datamining directly

In [ ]:
dosesPath = "./HNSCC/warpedDoses/"
availableDoses = ["HNSCC-01-{0}".format(a.split('.')[0]) for a in os.listdir(dosesPath)]

availablePatientsMask = selectedPatients['TCIA code'].isin(availableDoses)
probeDose = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(dosesPath, "{0:04d}.nii".format(int(2)))))

availablePatients = selectedPatients.loc[availablePatientsMask]
len(availablePatients)




doseArray = np.zeros((len(availablePatients), *probeDose.shape))
statusArray = np.zeros((len(availablePatients),))

print(doseArray.shape)


n = 0
for idx, pt in availablePatients.iterrows():
    dose_arr = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(dosesPath, f"{pt['TCIA code'].split('-')[-1]}.nii" ) ) )
    doseArray[n,...] = calculateVoxelwiseBED(dose_arr, pt["Number of Fractions"], alpha_beta=10.0)
    statusArray[n] = int(pt["Survival  (months)"] <= 12) ## survival @ 12 months
    n += 1

## Setting up for logistic regression

Logistic regression is a bit different to the other mining we have done so far, because it requires a matrix be built at every voxel, which also contains the clinical variables.

Before we can decide what to do about this, we need to figure out what variables we think should be included. We can hypothesis about this for now, but in reality it should be a multi-disciplinary discussion with an experienced clinician

For now, lets add

- Age at diagnosis ( a continuous variable)
- Surgery (a yes/no binary variable)
- Sex (Binary Male/Female)
- T stage (4 level factor) ???
- Dose (continuous ,in Gy, per voxel)

In the next cell, we organise the matrix that will contain these variables, and do the necessary transformations so that the logistic regression fitter will understand them


Note, after testing, I took the T stage out because it causes some nasty convergence issues (because it is so sparse). This is a problem when working with these types of data


In [ ]:
ages = availablePatients["Age"].values
surgery = (availablePatients["Surgery Summary"] != "No").values.astype(np.int32)
sex = (availablePatients["Sex"] =="Male").values.astype(np.int32)
encoder = OneHotEncoder(sparse=False)
tstage = encoder.fit_transform(availablePatients['T'].values.reshape((-1,1)), None)
clinicalVariableMatrix = np.zeros((len(statusArray), (4))) ## 4 from age, sex, surgery & dose
print(ages.shape)

## Now put things into the matrix
clinicalVariableMatrix[:,0] = ages
clinicalVariableMatrix[:,1] = surgery
clinicalVariableMatrix[:,2] = sex
# clinicalVariableMatrix[:,3:clinicalVariableMatrix.shape[-1]-1] = tstage
## Note we leave dose out for now

## See what the first row looks like...
print(clinicalVariableMatrix[0,:])


## Logistic regression loop functions

The logistic regression will be done per-voxel, meaning we want to do an independant regression in each one. We therefore need to loop over all the voxels, copy the doses into the last column of our matrix, then compute the logistic regression.

The output of the logistic regression will be a single image with a number of channels. The number of channels is determined by the number of variables. In our case, this will be 4. Let's write the loop now.

In [ ]:
def logistic_regression_per_voxel(doses, covariates, outcomes, mask=None):
    regressor = LogisticRegression(warm_start=True)
    flat_dose = doses.reshape((doses.shape[0], -1))
    result = np.zeros((flat_dose.shape[1], covariates.shape[-1]))
    
    if mask is None:
        ## loop on voxels
        for vox_idx in tqdm(range(flat_dose.shape[-1])):
            covariates[:,-1] = flat_dose[:,vox_idx] ## put dose into the regression table
            regressor.fit(covariates, outcomes)
            result[vox_idx,:] = regressor.coef_
        print(flat_dose.shape)
    else:
        for vox_loc in tqdm(np.argwhere(mask.reshape((-1,)) == 1)):
            covariates[:,-1] = flat_dose[:,vox_loc].squeeze() ## put dose into the regression table
            regressor.fit(covariates, outcomes)
            result[vox_loc,:] = regressor.coef_
    return result

We now have everything needed to run the per-voxel logistic regression. When I ran this on my laptop, it sounded like it would take off, and didn't really get anywhere!

In [ ]:
mask = sitk.GetArrayFromImage(sitk.ReadImage("./HNSCC/0002_mask.nii"))#.astype(np.float32)
print(np.argwhere(mask == 1))
res = logistic_regression_per_voxel(doseArray, clinicalVariableMatrix, statusArray, mask=mask)
# print(res.shape)

The above cell will take a very long time to run, because it is doing something fairly complicated on a large number of voxels. 


The next part of the workflow would be to run a permutation test to see if any of the covariates contain a significant region of interest. However, since this already took a long time to run one analysis, running 1000 or more is a prohibitively slow process which we won't do here.

Instead, I can show you how to produce the plots which might go into a publication

In [ ]:
fig, axarr = plt.subplots(2,2, figsize=(10,10))
print(axarr)
referenceAnatomy = sitk.GetArrayFromImage(sitk.ReadImage("./HNSCC/downsampledCTs/0002.nii"))[::-1,...]

slice_no = 40

res = res.reshape((*referenceAnatomy.shape, -1))

## Plot the reference CT on all axes
for idx, ax in enumerate(axarr.flatten()):
    ctImg = ax.imshow(referenceAnatomy[:,slice_no,:], cmap='Greys_r')
    channel = ax.imshow(res[::-1,slice_no,:,idx], alpha=0.5)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
fig.tight_layout()
    
    
    